# 7.2.2. Multi Layer Perception Model

1. See Notion Notes for short-hand notation, explanations, my thoughts, etc.
2. BOOKS:
    - Deep Learning for Time Series Forecasting - Predict the Future with MLPs, CNNs and LSTMs in Python by Jason Brownlee
    - Introduction to TSF with Python - How to Prepare Data and Develop Models to Predict the Future by Jason Brownlee

# Imports

In [1]:
# !cond

In [2]:
# import numpy as np 
from math import sqrt 
from numpy import array 
from numpy import mean 
from numpy import std 
from pandas import DataFrame 
from pandas import concat 
from pandas import read_csv 
from sklearn.metrics import mean_squared_error 
from keras.models import Sequential

from keras.models import Sequential 
from keras.layers import Dense
from matplotlib import pyplot

2023-05-29 18:56:25.020331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
BASE = '/Users/brinkley97/Documents/development/'
PATH_TO_BOOK = 'book-intro_to_tsf_with_python/'

In [4]:
# TS_AS_SML_FILE = BASE + PATH_TO_BOOK + '3-timeSeriesAsSupervisedLearning.ipynb'
# TS_AS_SML_FILE

In [5]:
# %load TS_AS_SML_FILE

In [6]:
# %run $TS_AS_SML_FILE

In [7]:
# series = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# n_steps = 3
# convert_uts_sequence_to_sml(series, n_steps)

# For Single Input

In [8]:
# model = Sequential()
# model.add(Dense(100, activation='relu' , input_dim=n_steps))
# model.add(Dense(1))
# model.compile(optimizer='adam' , loss='mse')

In [9]:
# # fit model 
# model.fit(X, y, epochs=2000, verbose=0) 
# # demonstrate prediction 
# x_input = array([70, 80, 90])
# x_input = x_input.reshape((1, n_steps)) 
# yhat = model.predict(x_input, verbose=0)
# print(yhat)

# For Full Datasets

In [10]:
# split a univariate dataset into train/test sets 
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [11]:
def series_to_supervised(data, n_in, n_out=1):
    
    df = DataFrame(data)
    cols = list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        
        # put it all together
        agg = concat(cols, axis=1) 
        
        # drop rows with NaN values
        agg.dropna(inplace=True)
    print(agg)
    return agg.values

In [12]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    # print("ACTUAL = ", actual)
    print("PREDICTED = ", predicted)
    return sqrt(mean_squared_error(actual, predicted))

In [13]:
# fit a model 
def model_fit(train, config): 
    # unpack config 
    n_input, n_nodes, n_epochs, n_batch = config 
    # prepare data 
    data = series_to_supervised(train, n_input) 
    print("data: ", data)
    train_x, train_y = data[:, :-1], data[:, -1] 
    # define model
    model = Sequential() 
    model.add(Dense(n_nodes, activation='relu' , input_dim=n_input)) 
    model.add(Dense(1)) 
    model.compile(loss='mse' , optimizer='adam' ) 
    
    # fit
    model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0) 
    print()
    
    return model

In [14]:
# forecast with a pre-fit model 
def model_predict(model, history, config): 
    # unpack config 
    n_input, _, _, _ = config 
    # print(n_input)
    # prepare data 
    # print(history)
    x_input = array(history[-n_input:]).reshape(1, n_input) 
    # print("x_input =", x_input)
    
    # forecast 
    yhat = model.predict(x_input, verbose=0) 
    # print(yhat)
    
    return yhat[0]

In [15]:
# walk-forward validation for univariate data 
def walk_forward_validation(data, n_test, cfg): 
    predictions = list()
    
    # split dataset 
    train, test = train_test_split(data, n_test) 
    print(train)
    # print(test)
    # fit model 
    model = model_fit(train, cfg) 
    # print(model)
    
    # seed history with training dataset 
    history = [x for x in train]
    # print(history)
    
    # step over each time-step in the test set 
    for i in range(len(test)): 
        # print(history)
        # fit model and make forecast for history 
        yhat = model_predict(model, history, cfg) 
        # print(yhat)
        
        # store forecast in list of predictions 
        predictions.append(yhat) 
        # print(i, predictions)
        
        # add actual observation to history for the next loop 
        # print(test[i])
        history.append(test[i])
        # print()
    # estimate prediction error 
    error = measure_rmse(test, predictions) 
    print('> %.3f' % error) 
    
    return error

In [16]:
# repeat evaluation of a config 
def repeat_evaluate(data, config, n_test, n_repeats=3): 
    # print(data)
    # fit and evaluate the model n times 
    scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
    print(scores)
    
    return scores

In [17]:
# summarize model performance
def summarize_scores(name, scores): 
    # print a summary
    scores_m, score_std = mean(scores), std(scores)
    print(' %s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std)) 
    
    # box and whisker plot 
    # pyplot.boxplot(scores)
    # pyplot.show()

In [18]:
dataset = BASE + PATH_TO_BOOK + 'datasets/daily-min-temperatures.csv'

In [19]:
series = read_csv(dataset, header=0, index_col=0)
data = series.values
# data split 
n_test = 12 

# define config
config = [24, 500, 100, 100] 
# grid search 
scores = repeat_evaluate(data, config, n_test)
# summarize scores 
# summarize_scores('mlp' , scores)

[[20.7]
 [17.9]
 [18.8]
 ...
 [13.9]
 [17.2]
 [14.7]]
         0     0     0     0     0     0     0     0     0     0  ...     0  \
24    20.7  17.9  18.8  14.6  15.8  15.8  15.8  17.4  21.8  20.0  ...  20.7   
25    17.9  18.8  14.6  15.8  15.8  15.8  17.4  21.8  20.0  16.2  ...  20.6   
26    18.8  14.6  15.8  15.8  15.8  17.4  21.8  20.0  16.2  13.3  ...  24.8   
27    14.6  15.8  15.8  15.8  17.4  21.8  20.0  16.2  13.3  16.7  ...  17.7   
28    15.8  15.8  15.8  17.4  21.8  20.0  16.2  13.3  16.7  21.5  ...  15.5   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
3633  15.0  12.2  10.5  11.1  13.0  12.9   8.8  14.7  14.7  12.7  ...  20.2   
3634  12.2  10.5  11.1  13.0  12.9   8.8  14.7  14.7  12.7  13.3  ...  19.4   
3635  10.5  11.1  13.0  12.9   8.8  14.7  14.7  12.7  13.3  13.2  ...  15.5   
3636  11.1  13.0  12.9   8.8  14.7  14.7  12.7  13.3  13.2  16.2  ...  14.1   
3637  13.0  12.9   8.8  14.7  14.7  12.7  13.3  13.2  16.2  17.3  ...  11.0  

2023-05-29 18:56:27.792299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



PREDICTED =  [array([14.212074], dtype=float32), array([15.051142], dtype=float32), array([13.652418], dtype=float32), array([13.997958], dtype=float32), array([14.780919], dtype=float32), array([12.133386], dtype=float32), array([14.749425], dtype=float32), array([15.018081], dtype=float32), array([13.774792], dtype=float32), array([14.070939], dtype=float32), array([13.989851], dtype=float32), array([15.218751], dtype=float32)]
> 1.783
[[20.7]
 [17.9]
 [18.8]
 ...
 [13.9]
 [17.2]
 [14.7]]
         0     0     0     0     0     0     0     0     0     0  ...     0  \
24    20.7  17.9  18.8  14.6  15.8  15.8  15.8  17.4  21.8  20.0  ...  20.7   
25    17.9  18.8  14.6  15.8  15.8  15.8  17.4  21.8  20.0  16.2  ...  20.6   
26    18.8  14.6  15.8  15.8  15.8  17.4  21.8  20.0  16.2  13.3  ...  24.8   
27    14.6  15.8  15.8  15.8  17.4  21.8  20.0  16.2  13.3  16.7  ...  17.7   
28    15.8  15.8  15.8  17.4  21.8  20.0  16.2  13.3  16.7  21.5  ...  15.5   
...    ...   ...   ...   ... 